In [1]:
#import necessary libraries
import matplotlib.pyplot as plt
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()
import keras


Instructions for updating:
non-resource variables are not supported in the long term


In [2]:
from keras.models import Sequential
from keras.layers import Dense, Activation, Flatten,Conv2D,MaxPooling2D
from keras.layers import LSTM, SimpleRNN

In [3]:
import numpy as np
a = open("data_shuffle_input.csv")
net_data_input = np.genfromtxt(a, delimiter=",")
b = open("data_shuffle_target.csv")
net_data_target = np.genfromtxt(b, delimiter=",")
c = open("mydata_shuffle_input.csv")
net_mydata_input = np.genfromtxt(c, delimiter=",")
d = open("mydata_shuffle_target.csv")
net_mydata_target = np.genfromtxt(d, delimiter=",")

In [4]:
import random
from sklearn.model_selection import cross_val_score
net_data_input = net_data_input / 255
net_data_input = net_data_input.reshape(net_data_input.shape[0],1,784)
net_mydata_input = net_mydata_input.reshape(net_mydata_input.shape[0],28,28,1)

print(net_data_input[0][0])


[0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         

In [5]:
acc_per_fold = []
loss_per_fold = []


In [6]:
num = []
for i in range(len(net_data_target)):
    if net_data_target[i][0] == 1:
        num.append(0)
    if net_data_target[i][1] == 1:
        num.append(1)
    if net_data_target[i][2] == 1:
        num.append(2)
    if net_data_target[i][3] == 1:
        num.append(3)
    if net_data_target[i][4] == 1:
        num.append(4)
    if net_data_target[i][5] == 1:
        num.append(5)
    if net_data_target[i][6] == 1:
        num.append(6)
    if net_data_target[i][7] == 1:
        num.append(7)
    if net_data_target[i][8] == 1:
        num.append(8)
    if net_data_target[i][9] == 1:
        num.append(9)
num = np.asarray(num)        
print(type(num))


<class 'numpy.ndarray'>


In [7]:
from sklearn.metrics import accuracy_score
from sklearn.model_selection import KFold

# Define the K-fold Cross Validator
kfold = KFold(n_splits=5)
acc_per_fold = []
acc = []
loss = []
# K-fold Cross Validation model evaluation
fold_no = 1
for train, test in kfold.split(net_data_input, net_data_target, net_mydata_input):

  
  # Define the model architecture for net mydata
    model = Sequential()

    model.add(Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_uniform', input_shape=(28, 28, 1)))
    model.add(MaxPooling2D((2, 2)))
    model.add(Flatten())
    model.add(Dense(100, activation='tanh', kernel_initializer='he_uniform'))
    model.add(Dense(10, activation='softmax'))
    # compile model
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
     # Generate a print
    # Fit data to model
    print('------------------------------------------------------------------------')
    print(f'Training for fold {fold_no} mydata')
    history_mydata = model.fit(net_mydata_input[train], net_data_target[train],
              batch_size=128,
              epochs=50,
              verbose=1)
    #print(f'Score for fold {fold_no}: {rnn_model.metrics_names[0]} of {scores[0]}; {rnn_model.metrics_names[1]} of {scores[1]*100}%')
    #acc_per_fold.append(scores[1] * 100)
    

    
    predictions_mydata_train = model.predict(net_mydata_input[train])
    classes_mydata_train = np.argmax(predictions_mydata_train, axis = 1)
    classes_mydata_train = tf.keras.utils.to_categorical(classes_mydata_train,10)

    predictions_mydata_test = model.predict(net_mydata_input[test])
    classes_mydata_test = np.argmax(predictions_mydata_test, axis = 1)
    classes_mydata_test = tf.keras.utils.to_categorical(classes_mydata_test,10)
    
    

    #make train list input
    joined_list_train = []
    joined_list_test = []

    for i in range(800):
        b = [*net_data_input[train][i][0], *classes_mydata_train[i]]
        joined_list_train.append(b)
        
    for j in range(200):
        a = [*net_data_input[test][j][0], *classes_mydata_test[j]]
        joined_list_test.append(a)

    joined_list_train = np.asarray(joined_list_train)
    joined_list_test = np.asarray(joined_list_test)
    
    
    joined_list_train = joined_list_train.reshape(joined_list_train.shape[0],1,794)

    joined_list_test = joined_list_test.reshape(joined_list_test.shape[0],1,794)
    
  # Define the model architecture for net total

    rnn_model_outputs = Sequential()
    #rnn_model.add(Flatten())
    rnn_model_outputs.add(LSTM(256,input_shape=joined_list_train[0].shape,activation="tanh"))
    #rnn_model.add(SimpleRNN(128,input_shape=xtrain[0].shape,return_sequences=True))
    #rnn_model.add(SimpleRNN(128))
    rnn_model_outputs.add(Dense(units=10,activation="softmax"))

  # Compile the model
    rnn_model_outputs.compile(optimizer='adam', loss='categorical_crossentropy',metrics=['accuracy'])


  # Generate a print
    print('------------------------------------------------------------------------')
    print(f'Training for fold {fold_no} input outputs')

  # Fit data to model
    history = rnn_model_outputs.fit(joined_list_train, net_data_target[train],
              batch_size=128,
              epochs=50,
              verbose=1)
    scores = rnn_model_outputs.evaluate(joined_list_test, net_data_target[test], verbose=0)
    #print(f'Score for fold {fold_no}: {rnn_model.metrics_names[0]} of {scores[0]}; {rnn_model.metrics_names[1]} of {scores[1]*100}%')
    #acc_per_fold.append(scores[1] * 100)
    loss_per_fold.append(scores[0])
    predictions = rnn_model_outputs.predict(joined_list_test)
    classes = np.argmax(predictions, axis = 1)
    print(num[test])
    acc_per_fold.append(accuracy_score(num[test],classes))
    
    print('accuracy is ' + str(accuracy_score(num[test],classes)))
    
  # Generate generalization metrics
    acc.append(history.history['acc'])
    loss.append(history.history['loss'])
    

  # Increase fold number
    fold_no = fold_no + 1

------------------------------------------------------------------------
Training for fold 1 mydata
Train on 800 samples
Epoch 1/50
800/800 [==============================] - 1s 2ms/sample - loss: 1.8314 - acc: 0.4288
Epoch 2/50
800/800 [==============================] - 0s 260us/sample - loss: 0.7900 - acc: 0.7713
Epoch 3/50
800/800 [==============================] - 0s 256us/sample - loss: 0.5805 - acc: 0.8700
Epoch 4/50
800/800 [==============================] - 0s 253us/sample - loss: 0.4846 - acc: 0.8963
Epoch 5/50
800/800 [==============================] - 0s 263us/sample - loss: 0.4016 - acc: 0.9038
Epoch 6/50
800/800 [==============================] - 0s 250us/sample - loss: 0.3490 - acc: 0.9150
Epoch 7/50
800/800 [==============================] - 0s 266us/sample - loss: 0.3071 - acc: 0.9312
Epoch 8/50
800/800 [==============================] - 0s 256us/sample - loss: 0.2822 - acc: 0.9325
Epoch 9/50
800/800 [==============================] - 0s 254us/sample - loss: 0.2545 - ac

D:\anaconda\lib\site-packages\keras\engine\training_v1.py:2359: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  updates=self.state_updates,


------------------------------------------------------------------------
Training for fold 1 input outputs
Train on 800 samples
Epoch 1/50
800/800 [==============================] - 1s 800us/sample - loss: 2.0916 - acc: 0.3887
Epoch 2/50
800/800 [==============================] - 0s 96us/sample - loss: 1.4378 - acc: 0.7738
Epoch 3/50
800/800 [==============================] - 0s 89us/sample - loss: 0.8730 - acc: 0.8450
Epoch 4/50
800/800 [==============================] - 0s 97us/sample - loss: 0.5523 - acc: 0.8863
Epoch 5/50
800/800 [==============================] - 0s 97us/sample - loss: 0.3845 - acc: 0.9150
Epoch 6/50
800/800 [==============================] - 0s 92us/sample - loss: 0.2893 - acc: 0.9325
Epoch 7/50
800/800 [==============================] - 0s 104us/sample - loss: 0.2259 - acc: 0.9488
Epoch 8/50
800/800 [==============================] - 0s 88us/sample - loss: 0.1774 - acc: 0.9625
Epoch 9/50
800/800 [==============================] - 0s 96us/sample - loss: 0.1434 - 

D:\anaconda\lib\site-packages\keras\engine\training_v1.py:2335: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  updates = self.state_updates


[1 3 9 2 6 9 5 4 8 7 1 5 2 6 7 1 5 6 8 1 1 5 9 4 9 4 6 6 9 8 7 0 3 7 3 4 7
 6 4 5 4 9 4 4 9 0 3 2 4 8 5 6 7 7 6 1 1 2 4 0 9 2 8 5 9 7 3 8 4 2 3 2 5 8
 4 5 4 8 8 5 9 0 4 2 9 8 0 2 5 3 6 1 2 6 5 9 1 6 0 6 4 1 8 4 6 9 7 0 6 4 7
 7 1 5 4 9 9 9 0 2 7 5 2 3 6 3 7 2 0 0 6 7 4 4 6 0 0 6 3 7 1 9 2 1 3 0 9 0
 7 4 6 5 9 0 9 3 0 1 7 8 8 7 4 4 1 1 3 1 2 9 6 4 5 2 1 7 8 2 5 4 1 1 6 5 8
 5 1 1 2 0 7 4 2 6 0 2 3 9 3 3]
accuracy is 0.925
------------------------------------------------------------------------
Training for fold 2 mydata
Train on 800 samples
Epoch 1/50
800/800 [==============================] - 0s 351us/sample - loss: 1.7869 - acc: 0.4350
Epoch 2/50
800/800 [==============================] - 0s 267us/sample - loss: 0.8188 - acc: 0.7550
Epoch 3/50
800/800 [==============================] - 0s 267us/sample - loss: 0.5691 - acc: 0.8600
Epoch 4/50
800/800 [==============================] - 0s 255us/sample - loss: 0.4663 - acc: 0.8850
Epoch 5/50
800/800 [==============================] - 0s 2

...

[1 0 2 3 7 2 6 0 2 8 5 1 7 8 6 3 1 3 1 3 5 5 0 8 7 5 9 9 2 5 6 5 7 6 3 8 6
 7 2 0 0 1 7 2 5 3 7 1 1 1 3 9 9 5 2 6 6 8 4 5 3 7 0 0 1 7 9 1 3 6 4 6 6 3
 5 4 2 9 2 9 6 3 2 7 6 0 1 2 5 0 0 7 6 2 8 5 3 8 8 8 4 4 4 6 0 8 8 9 9 3 4
 8 7 9 0 4 1 1 4 8 8 3 4 4 3 6 7 7 3 6 3 9 1 1 7 6 7 0 0 3 8 3 0 5 1 1 2 1
 8 3 6 7 2 1 7 5 0 7 9 7 1 0 8 2 3 6 3 6 4 2 6 6 1 9 3 6 9 4 7 4 2 4 0 8 5
 8 9 8 1 1 1 9 6 4 0 6 8 8 7 8]
accuracy is 0.93
------------------------------------------------------------------------
Training for fold 5 mydata
Train on 800 samples
Epoch 1/50
800/800 [==============================] - 0s 375us/sample - loss: 1.6819 - acc: 0.4725
Epoch 2/50
800/800 [==============================] - 0s 229us/sample - loss: 0.6744 - acc: 0.8000
Epoch 3/50
800/800 [==============================] - 0s 227us/sample - loss: 0.4941 - acc: 0.8675
Epoch 4/50
800/800 [==============================] - 0s 222us/sample - loss: 0.4001 - acc: 0.8913
Epoch 5/50
800/800 [==============================] - 0s 22

D:\anaconda\lib\site-packages\keras\engine\training_v1.py:2359: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  updates=self.state_updates,


------------------------------------------------------------------------
Training for fold 5 input outputs
Train on 800 samples
Epoch 1/50
800/800 [==============================] - 0s 406us/sample - loss: 2.1132 - acc: 0.4025
Epoch 2/50
800/800 [==============================] - 0s 108us/sample - loss: 1.4611 - acc: 0.7987
Epoch 3/50
800/800 [==============================] - 0s 107us/sample - loss: 0.8824 - acc: 0.8575
Epoch 4/50
800/800 [==============================] - 0s 120us/sample - loss: 0.5451 - acc: 0.8963
Epoch 5/50
800/800 [==============================] - 0s 119us/sample - loss: 0.3801 - acc: 0.9237
Epoch 6/50
800/800 [==============================] - 0s 119us/sample - loss: 0.2849 - acc: 0.9362
Epoch 7/50
800/800 [==============================] - 0s 110us/sample - loss: 0.2206 - acc: 0.9525
Epoch 8/50
800/800 [==============================] - 0s 117us/sample - loss: 0.1757 - acc: 0.9700
Epoch 9/50
800/800 [==============================] - 0s 111us/sample - loss: 0.

D:\anaconda\lib\site-packages\keras\engine\training_v1.py:2335: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  updates = self.state_updates


[6 6 6 1 8 0 6 9 9 2 0 3 6 7 1 3 3 5 8 0 5 7 2 3 9 0 5 7 2 5 2 3 8 5 8 0 8
 9 8 7 9 8 7 5 0 5 1 7 3 1 6 3 6 5 6 9 2 5 1 3 6 2 9 1 4 8 7 8 0 5 2 7 4 0
 2 9 0 2 2 7 5 8 8 2 2 9 6 9 4 2 0 0 5 7 3 9 1 8 5 1 1 9 0 4 2 0 1 4 0 1 2
 2 8 0 0 1 6 5 4 4 4 8 1 5 8 8 1 4 8 8 3 9 0 7 4 8 4 2 2 5 8 1 3 9 1 0 3 0
 8 3 4 9 2 4 0 6 8 0 2 9 9 5 7 1 7 2 3 5 7 5 1 8 5 0 1 2 1 5 1 4 8 4 0 2 9
 0 0 1 6 0 6 8 5 8 3 4 8 6 3 5]
accuracy is 0.91


In [8]:

# == Provide average scores ==
print('------------------------------------------------------------------------')
print('Score per fold')
for i in range(0, len(acc_per_fold)):
    
    print('------------------------------------------------------------------------')
    print(f'> Fold {i+1} - Loss: {loss_per_fold[i]} - Accuracy: {acc_per_fold[i]}%')
print('------------------------------------------------------------------------')
print('Average scores for all folds:')
print(f'> Accuracy: {np.mean(acc_per_fold)} (+- {np.std(acc_per_fold)})')
print(f'> Loss: {np.mean(loss_per_fold)}')
print('------------------------------------------------------------------------')

------------------------------------------------------------------------
Score per fold
------------------------------------------------------------------------
> Fold 1 - Loss: 0.3151681494805962 - Accuracy: 0.925%
------------------------------------------------------------------------
> Fold 2 - Loss: 0.49699158951640127 - Accuracy: 0.9%
------------------------------------------------------------------------
> Fold 3 - Loss: 0.3431153491139412 - Accuracy: 0.915%
------------------------------------------------------------------------
> Fold 4 - Loss: 0.26485167250037195 - Accuracy: 0.93%
------------------------------------------------------------------------
> Fold 5 - Loss: 0.42450804233551026 - Accuracy: 0.91%
------------------------------------------------------------------------
Average scores for all folds:
> Accuracy: 0.916 (+- 0.01067707825203132)
> Loss: 0.3689269605893642
------------------------------------------------------------------------


In [9]:
np.savetxt('CNN_input_output_cross_validation_acc.csv',acc, delimiter=',')

In [10]:
np.savetxt('CNN_input_output_cross_validation_loss.csv',loss, delimiter=',')